In [5]:
# packages
import pandas as pd
from bs4 import BeautifulSoup
import requests
from zipfile import ZipFile
import requests
import re
from datetime import date
import json
import xmltodict
import xml.etree.ElementTree as ET

In [33]:
# get links from website

URL = "https://ds.marktstammdatenregister.dev/Marktstammdatenregister"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
DataLink = soup.findAll('a', href = re.compile('^https://download.marktstammdatenregister.de/Gesamtdatenexport_'))[0]['href']

!!! Download takes ~15 minutes !!!

In [34]:
# download data
req = requests.get(DataLink)
# set filename
datum = date.today()
filename = f'MaStR_Gesamtdatenexport_{datum}.zip'
# writing file to system
print('Download started')
with open(filename, 'wb') as output_file:
    output_file.write(req.content)
print('Download completed')

Download started
Download completed


In [40]:
# unzip folder and extract required data
with ZipFile("/home/jan/Uni/DS-Project/modules/data/MaStR_Gesamtdatenexport_2022-11-06", 'r') as ZipObject:
    ZipObject.extract(
        "EinheitenSolar_1.xml", path = "/home/jan/Uni/DS-Project/data/MaStR/"
    )
    ZipObject.extract(
        "EinheitenWind.xml", path = "/home/jan/Uni/DS-Project/data/MaStR/"
    )
ZipObject.close()

In [7]:
# xml to data frame
WKA = "/home/jan/Uni/DS-Project/data/MaStR/EinheitenWind.xml"

# transform into python dictionary        
tree = ET.parse(WKA)
xml_data = tree.getroot()
#here you can change the encoding type to be able to set it to the one you need
xmlstr = ET.tostring(xml_data, encoding='utf-8', method='xml')

data_dict = dict(xmltodict.parse(xmlstr))

# transform into json
json_data = json.dumps(data_dict)
with open("WKA.json", "w") as json_file:
    json_file.write(json_data)